In [28]:
from nsetools import Nse
import yfinance as yf
import pandas as pd
import talib as ta
nse = Nse()

In [29]:
# Get all stock codes

all_stock_codes = nse.get_stock_codes()
keys = list(all_stock_codes.keys())[1:-1]
keys = [sub + '.NS' for sub in keys]

In [30]:
# List of strategy

# 1. Check 44ma rising(1d)
# 2. 44MA(1d,1w)
# 3. 44MA+200MA(1d,1w)
# 4. BollingerBand(1d,1w)

In [31]:
def get_data(tickers, period, interval):
    
    data = yf.download(tickers=tickers,period=period,interval=interval,progress=False)
    data['44MA'] = ta.SMA(data['Close'],44)
    data['200MA'] = ta.SMA(data['Close'],200)
    data['up_band'], data['mid_band'], data['low_band'] = ta.BBANDS(data['Close'], timeperiod =20)
    data = data.drop(['mid_band'],axis=1)
    
    return data

In [32]:
# Check if 44MA is rising for last n instances in dataframe: data

def check_44ma_rising(data,n):
    
    flag_44ma_rising = True
    for x in range(n):
        x +=1
        try:
            flag_44ma_rising = ((data['44MA'].iloc[-1*x] > data['44MA'].iloc[(-1*x)-1]) and flag_44ma_rising)
        except:
            flag_44ma_rising = False

    return flag_44ma_rising

In [33]:
def check_44ma_str(data):
    
    # Check 44MA increasing for 3 points
    flag_1 = check_44ma_rising(data,5)
    
    # Check for green candle
    if flag_1 == True:
        flag_2 = data['Close'].iloc[-1] > data['Open'].iloc[-1]
    else:
        flag_2 = False
    
    # Check for low to be close to bb_low_band
    if (flag_1 and flag_2) == True:
        flag_3 = (abs(data['Low'].iloc[-1] - data['44MA'].iloc[-1])/(data['44MA'].iloc[-1])) < 0.02
    else:
        flag_3 = False
    
    
    flag_44ma_str = ((flag_1 and flag_2) and flag_3)
    
    return flag_44ma_str

In [34]:
results = pd.DataFrame(columns=['Stock','44ma_rising'])
m = 0

for stock in keys:
    
    tickers = stock
    period = "3mo"
    interval = "1d"
    try:
        data = get_data(tickers,period,interval)
    except:
        continue
    
    # 44MA rising check    
#     flag_44ma_rising = check_44ma_rising(data,5)
#     if flag_44ma_rising==True:
#         results.loc[m]=[tickers,flag_44ma_rising]
#         m +=1
        
    # 44MA strategy check
    flag_44ma_str = check_44ma_str(data)
    if flag_44ma_str==True:
        results.loc[m]=[tickers,flag_44ma_str]
        m +=1
        
print(results)


1 Failed download:
- DIGJAMLMTD.NS: No data found for this date range, symbol may be delisted

1 Failed download:
- EMAMIPAP.NS: No data found for this date range, symbol may be delisted

1 Failed download:
- NIRAJISPAT.NS: No data found for this date range, symbol may be delisted

1 Failed download:
- SHAILY.NS: No data found, symbol may be delisted

1 Failed download:
- SUBCAPCITY.NS: No data found for this date range, symbol may be delisted

1 Failed download:
- VLIFEPP.NS: No data found, symbol may be delisted

1 Failed download:
- ZODIAC.NS: No data found for this date range, symbol may be delisted
            Stock  44ma_rising
0   ASIANHOTNR.NS         True
1   AUROPHARMA.NS         True
2     AXISBANK.NS         True
3          BLS.NS         True
4     CINELINE.NS         True
5   FEDERALBNK.NS         True
6   GENUSPAPER.NS         True
7        HIKAL.NS         True
8     HINDALCO.NS         True
9      HTMEDIA.NS         True
10    IMAGICAA.NS         True
11     INDIANB.N

In [9]:
data = yf.download(tickers='PIIND.NS',period="3mo",interval="1d",progress=False)
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,3035.000000,3108.000000,3007.000000,3051.750000,3048.165527,201183
2022-01-04,3050.000000,3062.399902,2997.649902,3013.899902,3010.359863,136534
2022-01-05,3027.250000,3051.850098,2961.199951,2993.199951,2989.684326,314219
2022-01-06,2970.000000,2988.899902,2930.000000,2954.300049,2950.830078,173758
2022-01-07,2969.800049,2969.800049,2916.800049,2949.050049,2945.586182,330943
...,...,...,...,...,...,...
2022-03-28,2814.000000,2819.699951,2722.199951,2750.000000,2750.000000,130688
2022-03-29,2761.000000,2864.500000,2753.350098,2837.600098,2837.600098,367845
2022-03-30,2850.000000,2862.949951,2795.000000,2851.250000,2851.250000,290427
